CREATING MOUNT POINT FOR INPUTCONTAINER

In [0]:
# Configuring the Azure Data Lake Storage details
#storage_account_name = "swiggyanalysisproject"  
#container_name = "inputcontainer"              
#storage_account_key = "aFy+oCL96bhKikOykNKFX4dveX1T+h6WeCktSD/EVf4sT7zN7NIJE0wfAv7SGYkvC33Sd/dy2FOe+AStqJqd6g=="  

# Mounting the storage
#dbutils.fs.mount( 
    #source=f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net/",
    #mount_point=f"/mnt/{container_name}",
    #extra_configs={f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net": storage_account_key}
#)


In [0]:
display(dbutils.fs.ls("/mnt/inputcontainer"))


path,name,size,modificationTime
dbfs:/mnt/inputcontainer/DIM.Date.Table.csv,DIM.Date.Table.csv,796835,1733929734000
dbfs:/mnt/inputcontainer/Swiggy_Analysis_Source_File.csv,Swiggy_Analysis_Source_File.csv,20259,1733929734000


In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("SwiggyAnalysis").getOrCreate()


In [0]:
import pyspark.sql.functions as F

file_path = "dbfs:/mnt/inputcontainer/Swiggy_Analysis_Source_File.csv"

# Reading the CSV file into a DataFrame
df = spark.read.format("csv").option("header", "true").load(file_path)

# Showing the first few rows of the DataFrame
df.show()

+--------------------+---------+--------------------+-------------+--------------------+-----------------+-------------------+----------------------+--------------+-----------+-----+
|     Restaurant Name|     City|            Locality|     Cuisines|Average Cost for two|Has Table booking|Has Online delivery| Rating Stars out of 5|Rating in text|Price range|Votes|
+--------------------+---------+--------------------+-------------+--------------------+-----------------+-------------------+----------------------+--------------+-----------+-----+
|    Sultans of Spice|Bangalore|BluPetal Hotel, K...| North Indian|                1300|              Yes|                Yes|                   4.1|     Very Good|          3|  314|
|The Fatty Bao - A...|Bangalore|         Indiranagar|        Asian|                2400|              Yes|                Yes|                   4.7|     Excellent|          4|  591|
|                Toit|Bangalore|         Indiranagar|      Italian|                20

DATA CLEANING

In [0]:
df.count()

258

In [0]:
df.show(50)

+--------------------+----------+--------------------+-------------+--------------------+-----------------+-------------------+----------------------+--------------+-----------+-----+
|     Restaurant Name|      City|            Locality|     Cuisines|Average Cost for two|Has Table booking|Has Online delivery| Rating Stars out of 5|Rating in text|Price range|Votes|
+--------------------+----------+--------------------+-------------+--------------------+-----------------+-------------------+----------------------+--------------+-----------+-----+
|    Sultans of Spice| Bangalore|BluPetal Hotel, K...| North Indian|                1300|              Yes|                Yes|                   4.1|     Very Good|          3|  314|
|The Fatty Bao - A...| Bangalore|         Indiranagar|        Asian|                2400|              Yes|                Yes|                   4.7|     Excellent|          4|  591|
|                Toit| Bangalore|         Indiranagar|      Italian|            

In [0]:
df.printSchema()

root
 |-- Restaurant Name: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Locality: string (nullable = true)
 |-- Cuisines: string (nullable = true)
 |-- Average Cost for two: string (nullable = true)
 |-- Has Table booking: string (nullable = true)
 |-- Has Online delivery: string (nullable = true)
 |--  Rating Stars out of 5: string (nullable = true)
 |-- Rating in text: string (nullable = true)
 |-- Price range: string (nullable = true)
 |-- Votes: string (nullable = true)



In [0]:
#Checking Null Values
from pyspark.sql.functions import col, sum
null_counts = df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns])
null_counts.show()

+---------------+----+--------+--------+--------------------+-----------------+-------------------+----------------------+--------------+-----------+-----+
|Restaurant Name|City|Locality|Cuisines|Average Cost for two|Has Table booking|Has Online delivery| Rating Stars out of 5|Rating in text|Price range|Votes|
+---------------+----+--------+--------+--------------------+-----------------+-------------------+----------------------+--------------+-----------+-----+
|              0|   0|       0|       0|                   0|                0|                  0|                     0|             0|          0|    0|
+---------------+----+--------+--------+--------------------+-----------------+-------------------+----------------------+--------------+-----------+-----+



In [0]:
df.distinct().count()

258

DATA TRANSFORMATION

In [0]:
# Verify column names and trim leading/trailing spaces
cleaned_columns = [col.strip() for col in df.columns]
df = df.toDF(*cleaned_columns)

# Print the cleaned column names to confirm
print(df.columns)

['Restaurant Name', 'City', 'Locality', 'Cuisines', 'Average Cost for two', 'Has Table booking', 'Has Online delivery', 'Rating Stars out of 5', 'Rating in text', 'Price range', 'Votes']


CREATING DIMENSION TABLES AND FACT TABLE

In [0]:
from pyspark.sql.functions import expr

city_dim = df.select("City").distinct() \
             .withColumnRenamed("City", "City_Name") \
             .withColumn("City_ID", expr("row_number() over (order by City_Name)"))

city_dim.show()


+----------+-------+
| City_Name|City_ID|
+----------+-------+
| Bangalore|      1|
|Chandigarh|      2|
|   Chennai|      3|
|       Goa|      4|
|  Guwahati|      5|
| Hyderabad|      6|
|    Jaipur|      7|
|   Kolkata|      8|
|   Lucknow|      9|
|    Mumbai|     10|
|     Patna|     11|
|      Pune|     12|
|     Surat|     13|
+----------+-------+



In [0]:
locality_dim = df.select("Locality").distinct() \
                 .withColumnRenamed("Locality", "Locality_Name") \
                 .withColumn("Locality_ID", expr("row_number() over (order by Locality_Name)"))

locality_dim.show()

+--------------------+-----------+
|       Locality_Name|Locality_ID|
+--------------------+-----------+
|12th Square Build...|          1|
|Acropolis Mall, K...|          2|
|          Adajan Gam|          3|
|               Adyar|          4|
|            Aminabad|          5|
|           Anandpuri|          6|
|Anil Plaza, Chris...|          7|
|              Anjuna|          8|
|Anjuna Beach, Anjuna|          9|
|     Anna Nagar East|         10|
|             Arambol|         11|
|         Ashok Nagar|         12|
|               Athwa|         13|
|          Azad Nagar|         14|
|                Baga|         15|
|Balewadi High Str...|         16|
|          Ballygunge|         17|
|               Baner|         18|
|       Banjara Hills|         19|
|          Betalbatim|         20|
+--------------------+-----------+
only showing top 20 rows



In [0]:
restaurant_dim = df.select("Restaurant Name").distinct() \
                   .withColumnRenamed("Restaurant Name", "Restaurant_Name") \
                   .withColumn("Restaurant_ID", expr("row_number() over (order by Restaurant_Name)"))

restaurant_dim.show()

+--------------------+-------------+
|     Restaurant_Name|Restaurant_ID|
+--------------------+-------------+
|11th Avenue Cafe ...|            1|
|      145 Kala Ghoda|            2|
|          17 Degrees|            3|
|18 Degrees Resto ...|            4|
|      38 Degree East|            5|
|           4 Seasons|            6|
|  6 Ballygunge Place|            7|
|AB's - Absolute B...|            8|
|    Agent Jack's Bar|            9|
|            Angeethi|           10|
|             Antares|           11|
|              Apache|           12|
|Asia Kitchen by M...|           13|
|        Baba Au Rhum|           14|
|         Bansi Vihar|           15|
|             Bar Bar|           16|
|               BarBQ|           17|
|     Barbeque Nation|           18|
|      Barbeque Ville|           19|
|  Basil With A Twist|           20|
+--------------------+-------------+
only showing top 20 rows



In [0]:
cuisine_dim = df.select("Cuisines").distinct() \
                .withColumnRenamed("Cuisines", "Cuisine_Name") \
                .withColumn("Cuisine_ID", expr("row_number() over (order by Cuisine_Name)"))

cuisine_dim.show()

+--------------+----------+
|  Cuisine_Name|Cuisine_ID|
+--------------+----------+
|      American|         1|
|         Asian|         2|
|        Bakery|         3|
|       Bengali|         4|
|       Biryani|         5|
|          Cafe|         6|
|Charcoal Grill|         7|
|       Chinese|         8|
|   Continental|         9|
|      Desserts|        10|
|      European|        11|
|     Fast Food|        12|
|          Goan|        13|
|      Gujarati|        14|
|  Healthy Food|        15|
|    Hyderabadi|        16|
|       Italian|        17|
|      Japanese|        18|
|      Lebanese|        19|
| Mediterranean|        20|
+--------------+----------+
only showing top 20 rows



In [0]:
table_booking_dim = df.select("Has Table booking").distinct() \
                      .withColumnRenamed("Has Table booking", "Table_booking_avail") \
                      .withColumn("Table_booking_ID", expr("row_number() over (order by Table_booking_avail)"))

table_booking_dim.show()

+-------------------+----------------+
|Table_booking_avail|Table_booking_ID|
+-------------------+----------------+
|                 No|               1|
|                Yes|               2|
+-------------------+----------------+



In [0]:
delivery_dim = df.select("Has Online delivery").distinct() \
                 .withColumnRenamed("Has Online delivery", "Delivery_avail") \
                 .withColumn("Delivery_ID", expr("row_number() over (order by Delivery_avail)"))

delivery_dim.show()

+--------------+-----------+
|Delivery_avail|Delivery_ID|
+--------------+-----------+
|            No|          1|
|           Yes|          2|
+--------------+-----------+



In [0]:
from pyspark.sql.functions import expr

rating_dim = df.selectExpr("`Rating Stars out of 5` as Rating_in_stars", "`Rating in text` as Rating_in_text").distinct() \
               .withColumn("Rating_ID", expr("row_number() over (order by Rating_in_stars)"))

rating_dim.show()


+---------------+--------------+---------+
|Rating_in_stars|Rating_in_text|Rating_ID|
+---------------+--------------+---------+
|            2.6|       Average|        1|
|            3.1|       Average|        2|
|            3.3|       Average|        3|
|            3.4|       Average|        4|
|            3.5|          Good|        5|
|            3.6|          Good|        6|
|            3.7|          Good|        7|
|            3.8|          Good|        8|
|            3.9|          Good|        9|
|              4|     Very Good|       10|
|            4.1|     Very Good|       11|
|            4.2|     Very Good|       12|
|            4.3|     Very Good|       13|
|            4.4|     Very Good|       14|
|            4.5|     Excellent|       15|
|            4.6|     Excellent|       16|
|            4.7|     Excellent|       17|
|            4.8|     Excellent|       18|
|            4.9|     Excellent|       19|
+---------------+--------------+---------+



In [0]:
df = df.toDF(*[col.strip().replace(" ", "_") for col in df.columns])
print(df.columns)

['Restaurant_Name', 'City', 'Locality', 'Cuisines', 'Average_Cost_for_two', 'Has_Table_booking', 'Has_Online_delivery', 'Rating_Stars_out_of_5', 'Rating_in_text', 'Price_range', 'Votes']


In [0]:
from pyspark.sql.functions import monotonically_increasing_id

fact_table = df \
    .join(city_dim, df["City"] == city_dim.City_Name, "inner") \
    .join(locality_dim, df["Locality"] == locality_dim.Locality_Name, "inner") \
    .join(restaurant_dim, df["Restaurant_Name"] == restaurant_dim.Restaurant_Name, "inner") \
    .join(cuisine_dim, df["Cuisines"] == cuisine_dim.Cuisine_Name, "inner") \
    .join(table_booking_dim, df["Has_Table_booking"] == table_booking_dim.Table_booking_avail, "inner") \
    .join(delivery_dim, df["Has_Online_delivery"] == delivery_dim.Delivery_avail, "inner") \
    .join(rating_dim, (df["Rating_Stars_out_of_5"] == rating_dim.Rating_in_stars) &
                     (df["Rating_in_text"] == rating_dim.Rating_in_text), "inner") \
    .selectExpr(
        "City_ID",
        "Locality_ID",
        "Restaurant_ID",
        "Cuisine_ID",
        "Average_Cost_for_two as Average_cost_for_two",
        "Rating_ID",
        "Delivery_ID",
        "Table_booking_ID",
        "Votes",
        "Price_range"
    )
display(fact_table)

City_ID,Locality_ID,Restaurant_ID,Cuisine_ID,Average_cost_for_two,Rating_ID,Delivery_ID,Table_booking_ID,Votes,Price_range
1,125,63,22,1500,14,1,1,392,3
1,110,83,9,1400,9,1,1,118,3
1,110,21,12,1800,15,2,2,211,3
1,105,51,9,1200,12,1,1,72,3
1,88,130,12,600,16,1,1,29,2
1,83,68,2,1200,14,1,2,223,3
1,83,8,11,1400,16,1,1,458,3
1,69,96,9,1500,15,1,1,294,3
1,68,59,24,400,13,2,1,488,1
1,67,182,24,1400,11,1,2,1070,3


In [0]:
# Saving dimension tables
city_dim.write.csv("city_dim.csv", header=True, mode="overwrite")
locality_dim.write.csv("locality_dim.csv", header=True, mode="overwrite")
restaurant_dim.write.csv("restaurant_dim.csv", header=True, mode="overwrite")
cuisine_dim.write.csv("cuisine_dim.csv", header=True, mode="overwrite")
table_booking_dim.write.csv("table_booking_dim.csv", header=True, mode="overwrite")
delivery_dim.write.csv("delivery_dim.csv", header=True, mode="overwrite")
rating_dim.write.csv("rating_dim.csv", header=True, mode="overwrite")

# Saving fact table
fact_table.write.csv("fact_swiggy.csv", header=True, mode="overwrite")


---------------------------------------------------------------------------
ExecutionError                            Traceback (most recent call last)
File <command-3993246892077466>, line 7
      4 storage_account_key = "aFy+oCL96bhKikOykNKFX4dveX1T+h6WeCktSD/EVf4sT7zN7NIJE0wfAv7SGYkvC33Sd/dy2FOe+AStqJqd6g=="  # Paste the storage account key here
      6 # Mount the storage
----> 7 dbutils.fs.mount(
      8     source=f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net/",
      9     mount_point=f"/mnt/{container_name}",
     10     extra_configs={f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net": storage_account_key}
     11 )

File /databricks/python_shell/dbruntime/dbutils.py:158, in prettify_exception_message.<locals>.f_with_exception_handling(*args, **kwargs)
    156 exc.__context__ = None
    157 exc.__cause__ = None
--> 158 raise exc

ExecutionError: An error occurred while calling o422.mount.
: java.rmi.RemoteException: java.lang.Illegal

CREATING MOUNT POINT FOR OUTPUTCONTAINER

In [0]:
# Configuring the Azure Data Lake Storage details
#storage_account_name = "swiggyanalysisproject"  
#container_name = "outputcontainer"              
#storage_account_key = "aFy+oCL96bhKikOykNKFX4dveX1T+h6WeCktSD/EVf4sT7zN7NIJE0wfAv7SGYkvC33Sd/dy2FOe+AStqJqd6g=="  

# Mounting the storage
#dbutils.fs.mount(
    #source=f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net/",
    #mount_point=f"/mnt/{container_name}",
    #extra_configs={f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net": storage_account_key}
#)

---------------------------------------------------------------------------
ExecutionError                            Traceback (most recent call last)
File <command-3993246892077466>, line 7
      4 storage_account_key = "aFy+oCL96bhKikOykNKFX4dveX1T+h6WeCktSD/EVf4sT7zN7NIJE0wfAv7SGYkvC33Sd/dy2FOe+AStqJqd6g=="  # Paste the storage account key here
      6 # Mount the storage
----> 7 dbutils.fs.mount(
      8     source=f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net/",
      9     mount_point=f"/mnt/{container_name}",
     10     extra_configs={f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net": storage_account_key}
     11 )

File /databricks/python_shell/dbruntime/dbutils.py:158, in prettify_exception_message.<locals>.f_with_exception_handling(*args, **kwargs)
    156 exc.__context__ = None
    157 exc.__cause__ = None
--> 158 raise exc

ExecutionError: An error occurred while calling o422.mount.
: java.rmi.RemoteException: java.lang.Illegal

 Count of Restaurant city wise

In [0]:
restaurant_city_count = fact_table \
    .join(city_dim, fact_table["City_ID"] == city_dim["City_ID"], "inner") \
    .groupBy("City_Name") \
    .count() \
    .withColumnRenamed("count", "Restaurant_Count")

# Showing the results
restaurant_city_count.show()

+----------+----------------+
| City_Name|Restaurant_Count|
+----------+----------------+
| Bangalore|              20|
|Chandigarh|              18|
|   Chennai|              20|
|       Goa|              20|
|  Guwahati|              21|
| Hyderabad|              18|
|    Jaipur|              20|
|   Kolkata|              20|
|   Lucknow|              21|
|    Mumbai|              20|
|     Patna|              20|
|      Pune|              20|
|     Surat|              20|
+----------+----------------+



In [0]:
#restaurant_city_count.write.format("csv") \
#  .option("header", True) \
#  .save(f"/mnt/outputcontainer/restaurant_city_count.csv")

---------------------------------------------------------------------------
ExecutionError                            Traceback (most recent call last)
File <command-3993246892077466>, line 7
      4 storage_account_key = "aFy+oCL96bhKikOykNKFX4dveX1T+h6WeCktSD/EVf4sT7zN7NIJE0wfAv7SGYkvC33Sd/dy2FOe+AStqJqd6g=="  # Paste the storage account key here
      6 # Mount the storage
----> 7 dbutils.fs.mount(
      8     source=f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net/",
      9     mount_point=f"/mnt/{container_name}",
     10     extra_configs={f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net": storage_account_key}
     11 )

File /databricks/python_shell/dbruntime/dbutils.py:158, in prettify_exception_message.<locals>.f_with_exception_handling(*args, **kwargs)
    156 exc.__context__ = None
    157 exc.__cause__ = None
--> 158 raise exc

ExecutionError: An error occurred while calling o422.mount.
: java.rmi.RemoteException: java.lang.Illegal

City-wise Top 10 restaurant based on Avg cost for 2

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Creating a window spec to partition by City and order by Average Cost for Two
window_spec = Window.partitionBy("City").orderBy(F.col("Average_Cost_for_two").desc())

# Adding a rank column based on the window specification
df_with_rank = df.withColumn("Rank", F.row_number().over(window_spec))

# Filtering to get only top 10 restaurants for each city
top_10_restaurants_city_wise = df_with_rank.filter(F.col("Rank") <= 10)

# Selecting relevant columns and show the result
top_10_restaurants_city_wise = top_10_restaurants_city_wise.select(
    "City",
    "Restaurant_Name",  # Corrected column name
    "Average_Cost_for_two",  # Corrected column name
    "Rank"
)

# Showing the result
top_10_restaurants_city_wise.show()

+----------+--------------------+--------------------+----+
|      City|     Restaurant_Name|Average_Cost_for_two|Rank|
+----------+--------------------+--------------------+----+
| Bangalore|  ECHOES Koramangala|                 950|   1|
| Bangalore|    Glen's Bakehouse|                 800|   2|
| Bangalore|            Truffles|                 800|   3|
| Bangalore|              Onesta|                 600|   4|
| Bangalore|              Onesta|                 600|   5|
| Bangalore|              Onesta|                 600|   6|
| Bangalore|          Eat Street|                 400|   7|
| Bangalore|The Fatty Bao - A...|                2400|   8|
| Bangalore|                Toit|                2000|   9|
| Bangalore|         Big Brewsky|                1800|  10|
|Chandigarh|   The Night Factory|                 850|   1|
|Chandigarh|             Karim's|                 800|   2|
|Chandigarh|           Pal Dhaba|                 700|   3|
|Chandigarh|         Nik Baker's|       

In [0]:
#top_10_restaurants_city_wise.write.format("csv") \
#  .option("header", True) \
#  .save(f"/mnt/outputcontainer/top_10_restaurants_city_wise.csv")

---------------------------------------------------------------------------
ExecutionError                            Traceback (most recent call last)
File <command-3993246892077466>, line 7
      4 storage_account_key = "aFy+oCL96bhKikOykNKFX4dveX1T+h6WeCktSD/EVf4sT7zN7NIJE0wfAv7SGYkvC33Sd/dy2FOe+AStqJqd6g=="  # Paste the storage account key here
      6 # Mount the storage
----> 7 dbutils.fs.mount(
      8     source=f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net/",
      9     mount_point=f"/mnt/{container_name}",
     10     extra_configs={f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net": storage_account_key}
     11 )

File /databricks/python_shell/dbruntime/dbutils.py:158, in prettify_exception_message.<locals>.f_with_exception_handling(*args, **kwargs)
    156 exc.__context__ = None
    157 exc.__cause__ = None
--> 158 raise exc

ExecutionError: An error occurred while calling o422.mount.
: java.rmi.RemoteException: java.lang.Illegal

 Top 10 restaurant based on Avg Votes

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Creating a window spec to partition by City and order by Votes (in descending order)
window_spec_votes = Window.partitionBy("City").orderBy(F.col("Votes").desc())

# Adding a rank column based on the window specification
df_with_rank_votes = df.withColumn("Rank", F.row_number().over(window_spec_votes))

# Filtering to get only top 10 restaurants for each city based on Votes
top_10_restaurants_votes = df_with_rank_votes.filter(F.col("Rank") <= 10)

# Selecting relevant columns and show the result
top_10_restaurants_votes = top_10_restaurants_votes.select(
    "City",
    "Restaurant_Name",  # Corrected column name
    "Votes",  # Corrected column name
    "Rank"
)

# Showing the result
top_10_restaurants_votes.show()

+----------+--------------------+-----+----+
|      City|     Restaurant_Name|Votes|Rank|
+----------+--------------------+-----+----+
| Bangalore|           Communiti|   72|   1|
| Bangalore|              Onesta|  677|   2|
| Bangalore|  ECHOES Koramangala|  621|   3|
| Bangalore|The Fatty Bao - A...|  591|   4|
| Bangalore|            Truffles|  532|   5|
| Bangalore|              Onesta|  520|   6|
| Bangalore|          Eat Street|  488|   7|
| Bangalore|AB's - Absolute B...|  458|   8|
| Bangalore|          Farzi Cafe|  392|   9|
| Bangalore| Three Dots & A Dash|  365|  10|
|Chandigarh|    Kylin Experience|    9|   1|
|Chandigarh|        Super Donuts|    9|   2|
|Chandigarh|           Taco Bell|    9|   3|
|Chandigarh|           Pal Dhaba|    8|   4|
|Chandigarh|    Brooklyn Central|  618|   5|
|Chandigarh|             Chili's|    6|   6|
|Chandigarh|   The Night Factory|  535|   7|
|Chandigarh|       Midnight Chef|    5|   8|
|Chandigarh|            Burgrill|    5|   9|
|Chandigar

In [0]:
#top_10_restaurants_votes.write.format("csv") \
#  .option("header", True) \
#  .save(f"/mnt/outputcontainer/top_10_restaurants_votes.csv")

---------------------------------------------------------------------------
ExecutionError                            Traceback (most recent call last)
File <command-3993246892077466>, line 7
      4 storage_account_key = "aFy+oCL96bhKikOykNKFX4dveX1T+h6WeCktSD/EVf4sT7zN7NIJE0wfAv7SGYkvC33Sd/dy2FOe+AStqJqd6g=="  # Paste the storage account key here
      6 # Mount the storage
----> 7 dbutils.fs.mount(
      8     source=f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net/",
      9     mount_point=f"/mnt/{container_name}",
     10     extra_configs={f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net": storage_account_key}
     11 )

File /databricks/python_shell/dbruntime/dbutils.py:158, in prettify_exception_message.<locals>.f_with_exception_handling(*args, **kwargs)
    156 exc.__context__ = None
    157 exc.__cause__ = None
--> 158 raise exc

ExecutionError: An error occurred while calling o422.mount.
: java.rmi.RemoteException: java.lang.Illegal

Top 10 restaurant based on Rating City wise

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Creating a window spec to partition by City and order by Rating_Stars_out_of_5 (in descending order)
window_spec_rating = Window.partitionBy("City").orderBy(F.col("Rating_Stars_out_of_5").desc())

# Adding a rank column based on the window specification
df_with_rank_rating = df.withColumn("Rank", F.row_number().over(window_spec_rating))

# Filtering to get only top 10 restaurants for each city based on Rating
top_10_restaurants_rating = df_with_rank_rating.filter(F.col("Rank") <= 10)

# Selecting relevant columns and showing the result
top_10_restaurants_rating = top_10_restaurants_rating.select(
    "City",
    "Restaurant_Name",  # Correcting column name for restaurant name
    "Rating_Stars_out_of_5",  # Correcting column name for rating
    "Rank"
)

# Showing the result
top_10_restaurants_rating.show()

+----------+--------------------+---------------------+----+
|      City|     Restaurant_Name|Rating_Stars_out_of_5|Rank|
+----------+--------------------+---------------------+----+
| Bangalore|                Toit|                  4.8|   1|
| Bangalore|The Fatty Bao - A...|                  4.7|   2|
| Bangalore|  ECHOES Koramangala|                  4.7|   3|
| Bangalore|            Truffles|                  4.7|   4|
| Bangalore|              Onesta|                  4.6|   5|
| Bangalore|AB's - Absolute B...|                  4.6|   6|
| Bangalore|              Onesta|                  4.6|   7|
| Bangalore|  Koramangala Social|                  4.5|   8|
| Bangalore|         Big Brewsky|                  4.5|   9|
| Bangalore|            Flechazo|                  4.4|  10|
|Chandigarh|     Barbeque Nation|                  4.5|   1|
|Chandigarh|            Burgrill|                  4.5|   2|
|Chandigarh|    Virgin Courtyard|                  4.4|   3|
|Chandigarh|            

In [0]:
#top_10_restaurants_rating.write.format("csv") \
#  .option("header", True) \
#  .save(f"/mnt/outputcontainer/top_10_restaurants_rating.csv")

---------------------------------------------------------------------------
ExecutionError                            Traceback (most recent call last)
File <command-3993246892077466>, line 7
      4 storage_account_key = "aFy+oCL96bhKikOykNKFX4dveX1T+h6WeCktSD/EVf4sT7zN7NIJE0wfAv7SGYkvC33Sd/dy2FOe+AStqJqd6g=="  # Paste the storage account key here
      6 # Mount the storage
----> 7 dbutils.fs.mount(
      8     source=f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net/",
      9     mount_point=f"/mnt/{container_name}",
     10     extra_configs={f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net": storage_account_key}
     11 )

File /databricks/python_shell/dbruntime/dbutils.py:158, in prettify_exception_message.<locals>.f_with_exception_handling(*args, **kwargs)
    156 exc.__context__ = None
    157 exc.__cause__ = None
--> 158 raise exc

ExecutionError: An error occurred while calling o422.mount.
: java.rmi.RemoteException: java.lang.Illegal

 Rating based on delivery availability City wise

In [0]:
df = df.withColumn("Delivery_Availability", 
                   F.when(F.col("Has_Online_delivery") == "Yes", "Yes")
                    .otherwise("No"))

# Filtering restaurants with delivery availability
delivery_available = df.filter(F.col("Delivery_Availability") == "Yes")

# Grouping by City and calculating the average rating
city_delivery_ratings = delivery_available.groupBy("City") \
    .agg(
        F.avg("Rating_Stars_out_of_5").alias("Average_Rating")
    ) \
    .orderBy(F.col("Average_Rating").desc())

# Adding a column showing that delivery is available
city_delivery_ratings = city_delivery_ratings.withColumn(
    "Delivery_Available", F.lit("Yes")
)

# Showing the result
city_delivery_ratings.show()


+----------+------------------+------------------+
|      City|    Average_Rating|Delivery_Available|
+----------+------------------+------------------+
| Bangalore|               4.4|               Yes|
| Hyderabad|               4.3|               Yes|
|    Jaipur|4.2700000000000005|               Yes|
|   Chennai| 4.269230769230768|               Yes|
|   Kolkata|4.2250000000000005|               Yes|
|      Pune| 4.171428571428572|               Yes|
|    Mumbai|4.1571428571428575|               Yes|
|Chandigarh| 4.016666666666667|               Yes|
+----------+------------------+------------------+



In [0]:
#city_delivery_ratings.write.format("csv") \
#  .option("header", True) \
#  .save(f"/mnt/outputcontainer/city_delivery_ratings.csv")

Avg cost for 2 based on cuisine city wise

In [0]:
avg_cost_city_cuisine = df.groupBy("City", "Cuisines") \
    .agg(F.avg("Average_Cost_for_two").alias("Average_Cost_for_Two")) \
    .orderBy("City", F.col("Average_Cost_for_Two").desc())

# Showing the result
avg_cost_city_cuisine.show()


+----------+-------------+--------------------+
|      City|     Cuisines|Average_Cost_for_Two|
+----------+-------------+--------------------+
| Bangalore|      Italian|              2000.0|
| Bangalore|        Asian|              1800.0|
| Bangalore|Modern Indian|              1500.0|
| Bangalore|     European|              1350.0|
| Bangalore|  Continental|              1262.5|
| Bangalore| North Indian|  1033.3333333333333|
| Bangalore|    Fast Food|               900.0|
| Bangalore|       Bakery|               800.0|
| Bangalore|     American|               800.0|
|Chandigarh|      Italian|              2200.0|
|Chandigarh|     American|              1700.0|
|Chandigarh|     Japanese|              1600.0|
|Chandigarh|  Continental|              1400.0|
|Chandigarh| North Indian|              1010.0|
|Chandigarh|      Mexican|              1000.0|
|Chandigarh|      Mughlai|               800.0|
|Chandigarh|         Cafe|               650.0|
|Chandigarh|       Bakery|              

In [0]:
#avg_cost_city_cuisine.write.format("csv") \
#  .option("header", True) \
#  .save(f"/mnt/outputcontainer/avg_cost_city_cuisine.csv")

SQL OPERATIONS

In [0]:
# Connection details for Azure SQL Database jdbc:sqlserver://swiggyproject.database.windows.net;databaseName=swiggyDB
jdbc_url = "jdbc:sqlserver://swiggyproject.database.windows.net:1433;database=swiggyDB"
connection_properties = {
    "user": "anshay",
    "password": "uCe5FrshDT@Gb6a",
    "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

# Save each DataFrame to Azure SQL Database
restaurant_city_count.write.jdbc(
    url=jdbc_url,
    table="restaurant_city_count",
    mode="overwrite",  # Options: 'append', 'overwrite', 'ignore', 'error'
    properties=connection_properties
)

top_10_restaurants_city_wise.write.jdbc(
    url=jdbc_url,
    table="top_10_restaurants_city_wise",
    mode="overwrite",
    properties=connection_properties
)

top_10_restaurants_votes.write.jdbc(
    url=jdbc_url,
    table="top_10_restaurants_votes",
    mode="overwrite",
    properties=connection_properties
)

top_10_restaurants_rating.write.jdbc(
    url=jdbc_url,
    table="top_10_restaurants_rating",
    mode="overwrite",
    properties=connection_properties
)

city_delivery_ratings.write.jdbc(
    url=jdbc_url,
    table="city_delivery_ratings",
    mode="overwrite",
    properties=connection_properties
)

avg_cost_city_cuisine.write.jdbc(
    url=jdbc_url,
    table="avg_cost_city_cuisine",
    mode="overwrite",
    properties=connection_properties
)